## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-09-08-59-17 +0000,nyt,U.S. Border Patrol Agents Shoot 2 in Portland ...,https://www.nytimes.com/2026/01/08/us/politics...
1,2026-01-09-08-58-02 +0000,nyt,"What We Know About the Shooting in Portland, Ore.",https://www.nytimes.com/2026/01/09/us/portland...
2,2026-01-09-08-49-00 +0000,wsj,Gold Rises Ahead of Key U.S. Economic Data,https://www.wsj.com/finance/commodities-future...
3,2026-01-09-08-42-00 +0000,wsj,Oil Gains Further With Focus on Geopolitical R...,https://www.wsj.com/finance/commodities-future...
4,2026-01-09-08-39-41 +0000,bbc,Elon Musk's Grok AI image editing limited to p...,https://www.bbc.com/news/articles/c99kn52nx9do...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2338/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
82,trump,82
76,venezuela,34
357,ice,32
391,new,22
7,shooting,21


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
106,2026-01-09-00-09-00 +0000,wsj,President Trump lashed out at five GOP senator...,https://www.wsj.com/politics/policy/senate-vot...,192
155,2026-01-08-22-40-00 +0000,wsj,The architect of a new White House ballroom to...,https://www.wsj.com/politics/policy/white-hous...,165
277,2026-01-08-17-23-16 +0000,nyt,"After ICE Shooting, Trump Administration Deplo...",https://www.nytimes.com/2026/01/08/us/politics...,164
121,2026-01-08-23-47-49 +0000,nyt,"Trump Addresses Venezuela, Greenland and Presi...",https://www.nytimes.com/2026/01/08/us/politics...,160
180,2026-01-08-21-37-00 +0000,wsj,President Trump demonstrated the tight grip he...,https://www.wsj.com/politics/policy/trump-flex...,150


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
106,192,2026-01-09-00-09-00 +0000,wsj,President Trump lashed out at five GOP senator...,https://www.wsj.com/politics/policy/senate-vot...
355,87,2026-01-08-11-57-00 +0000,wsj,ICE shooting creates new crack in a fractured ...,https://www.wsj.com/politics/policy/ice-shooti...
155,61,2026-01-08-22-40-00 +0000,wsj,The architect of a new White House ballroom to...,https://www.wsj.com/politics/policy/white-hous...
284,57,2026-01-08-16-59-04 +0000,nypost,Wealth tax threat prompts at least six billion...,https://nypost.com/2026/01/08/business/wealth-...
124,50,2026-01-08-23-40-30 +0000,nypost,Democrats dig-in on anti-ICE rhetoric as prote...,https://nypost.com/2026/01/08/us-news/democrat...
260,46,2026-01-08-18-13-50 +0000,nypost,Russia rejects Trump’s 20-point peace plan for...,https://nypost.com/2026/01/08/us-news/russia-r...
149,43,2026-01-08-22-46-01 +0000,nyt,Bondi Warns Minnesota Protesters to Stay Away ...,https://www.nytimes.com/2026/01/08/us/politics...
123,43,2026-01-08-23-42-00 +0000,wsj,A middle-of-the-night deal back in early July ...,https://www.wsj.com/politics/policy/aca-vote-h...
54,37,2026-01-09-03-08-27 +0000,nypost,Trump plans to meet Venezuelan opposition lead...,https://nypost.com/2026/01/08/us-news/trump-pl...
180,37,2026-01-08-21-37-00 +0000,wsj,President Trump demonstrated the tight grip he...,https://www.wsj.com/politics/policy/trump-flex...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
